# Serve Statistics Notebook

### Load Packages

In [ ]:
import pandas as pd
import numpy as np
import os
import json
import ipywidgets as widgets
from IPython.display import display

### Create Data Folder

In [ ]:
def create_data_folder():
    # Define the folder name
    folder_name = "data"
    
    # Check if the folder exists
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print(f"Folder '{folder_name}' created.")
    else:
        print(f"Folder '{folder_name}' already exists.")

In [ ]:
create_data_folder()

### Read Data

In [ ]:
# Dropdown with a non-valid default option
choose_player_dropdown = widgets.Dropdown(
    options=['-- Select --', 'Rudy Quan', 'Emon Van Loben Sels', 'Kaylan Bigun', 'Alexander Hoogmartens', 
             'Spencer Johnson', 'Aadarsh Tripathi', 'Giacomo Revelli', 'Gianluca Ballotta'],
    value='-- Select --',
    description='Category:'
)

display(choose_player_dropdown)

In [ ]:
# Check selection before proceeding
if choose_player_dropdown.value == '-- Select --':
    raise ValueError("Please choose a valid category from the dropdown menu in the previous cell before proceeding.")

# If valid, use the value
player_name = choose_player_dropdown.value

In [ ]:
combined_data_shots = pd.read_excel(f'../../data/mens/{player_name}/combined.xlsx', sheet_name='Shots')
combined_data_points = pd.read_excel(f'../../data/mens/{player_name}/combined.xlsx', sheet_name='Points')
combined_data_games = pd.read_excel(f'../../data/mens/{player_name}/combined.xlsx', sheet_name='Games')
combined_data_sets = pd.read_excel(f'../../data/mens/{player_name}/combined.xlsx', sheet_name='Sets')
combined_data_stats = pd.read_excel(f'../../data/mens/{player_name}/combined.xlsx', sheet_name='Stats')

### Average Service Game Duration

In [ ]:
def average_service_time(data):

    # Use combined_data_games 
    # Subset 'Server' column name for only host (host is always UCLA player)
    # find the mean of the 'Duration' Column

    avg_seconds = data[data['Server'] == 'host']['Duration'].mean() # Automatically coerces NA
    total = int(round(avg_seconds))
    mins, secs = divmod(total, 60)    

    return f"{mins}:{secs:02d}"

In [ ]:
# Output Average Service Game Duration
avg_service_game_duration = average_service_time(combined_data_games)
min, sec = avg_service_game_duration.split(':')
min, sec

### Average Games Held Percentage

In [ ]:
def service_games_won_percentage(df):
    
    # Subset Dataframe to only be UCLA Player serving
    service_games = df[df["Server"] == "host"]

    # Subset to only complete games
    service_games = service_games[service_games["Game Winner"] != "draw"]

    # Find the percentage of the "Game Winner" column everytime the value is "host"
    percentage = service_games["Game Winner"].value_counts(normalize=True).get('host', 0) * 100

    # Round and make number into an integer
    percentage = int(round(percentage, 0))

    return percentage

In [ ]:
average_games_held = str(service_games_won_percentage(combined_data_games)) + '%'
average_games_held

### Average Breakpoints Saved

In [ ]:
def breakpoints_saved_function(data):

    # Filter Data
    filtered_data = data[(data['Match Server'] == 'host') & 
                         (data['Break Point'] == True)
                         ].copy()

    percentage = int(round(filtered_data['Point Winner'].value_counts(normalize=True).get('host', 0) * 100, 0))

    return percentage

In [ ]:
breakpoints_saved_percentage = str(breakpoints_saved_function(combined_data_points)) + '%'
breakpoints_saved_percentage

### Average Aces

In [ ]:
def average_aces(df):

    # Filter for the row where 'Stat Name' is 'Aces'
    aces_row = df[df['Stat Name'] == 'Aces']

    if aces_row.empty:
        print("No 'Aces' row found.")
        return None
    
    # Columns that contain the per-set values
    set_columns = [col for col in df.columns if 'Host Set' in col]

    # Extract ace counts per match from those columns
    aces_per_match = aces_row[set_columns].sum(axis=1)
    
    # Calculate and return the average
    average = aces_per_match.mean()
    return round(average, 1)

In [ ]:
# Output Average Aces
average_aces = average_aces(combined_data_stats)
average_aces

### Average Double Faults

In [ ]:
def average_doubleFaults(df):

    # Filter only rows with Stat Name = '2nd Serves' and '2nd Serves In'
    second_serves = df[df['Stat Name'].str.strip() == '2nd Serves'].copy()
    second_serves_in = df[df['Stat Name'].str.strip() == '2nd Serves In'].copy()

    set_columns = [col for col in df.columns if 'Host Set' in col]

    second_serves_vals = second_serves[set_columns].sum(axis=1).reset_index(drop=True)
    second_serves_in_vals = second_serves_in[set_columns].sum(axis=1).reset_index(drop=True)
    average_double_faults = (second_serves_vals - second_serves_in_vals).mean()

    # Return average
    return round(average_double_faults, 1)

In [ ]:
average_double_faults = average_doubleFaults(combined_data_stats)
average_double_faults

### Serve In/Won Percentages

In [ ]:
# Helper Function
def find_stat(df, stat_name):
    # Subset to only get rows of specified Statistic
    stat_total = df.loc[df['Stat Name'] == stat_name]

    # Grab column names
    column_names = stat_total.columns 

    # Subset column names that only start with 'Host Set'
    column_names_subset = column_names[column_names.str.startswith('Host Set')]

    # 
    stat_total_value = stat_total[column_names_subset].sum().sum()
    return stat_total_value

In [ ]:
first_serves = find_stat(combined_data_stats, '1st Serves')   
first_serves_in = find_stat(combined_data_stats, '1st Serves In') 
first_serves_won = find_stat(combined_data_stats, '1st Serves Won')   

second_serves = find_stat(combined_data_stats, '2nd Serves')   
second_serves_in = find_stat(combined_data_stats, '2nd Serves In') 
second_serves_won = find_stat(combined_data_stats, '2nd Serves Won')   

In [ ]:
first_serve_in_percentage = int(round((first_serves_in / first_serves) * 100, 0))
first_serve_won_percentage = int(round((first_serves_won / first_serves_in) * 100, 0))
second_serve_in_percentage = int(round((second_serves_in / second_serves) * 100, 0))
second_serve_won_percentage = int(round((second_serves_won / second_serves_in) * 100, 0))

print(f"Serve Performance Summary for {player_name}:\n")
print(f"  1st Serve In %:        {first_serve_in_percentage}%")
print(f"  1st Serve Won %:       {first_serve_won_percentage}%")
print(f"  2nd Serve In %:        {second_serve_in_percentage}%")
print(f"  2nd Serve Won %:       {second_serve_won_percentage}%")


### Total Serve Points Won

In [ ]:
total_serves = combined_data_points[combined_data_points['Match Server'] == 'host'].shape[0]
# total_serves = find_stat(combined_data_stats, '1st Serves') # OR this too
serve_points_won = combined_data_points[combined_data_points['Match Server'] == 'host']['Point Winner'].value_counts().get(0, 'host')

total_serve_points_won = int(round((serve_points_won / total_serves) * 100, 0))
total_serve_points_won

##### Output Json

In [ ]:
# Build dictionary
serve_json = {
    "type": "Total Serve Points Won",
    "total": int(total_serves),
    "won": int(serve_points_won)
}

# Write to JSON
with open("data/total_serves.json", "w") as f:
    json.dump(serve_json, f, indent=4)

print("Saved to data/total_serves.json")

### Fastest Serve

In [ ]:
def fastest_serve(data):
    max_serve = data[data['Type'].isin(['first_serve', 'second_serve'])]['Speed (MPH)'].max()
    return int(round(max_serve, 0))

In [ ]:
fastest_serve = fastest_serve(combined_data_shots)
fastest_serve

### Favorite Serve

##### Helper Function

In [ ]:
# Helper Function: Classify Zones borrowed from swingvison_transformation.ipynb

def classify_zone(df):
    x = df['x_coord']
    y = df['y_coord']
    sign = x * y # if sign is pos, it's on ad side, if neg, it's deuce

    if (x < -105) or (x > 105):
        if sign > 0:
            side, zone = 'Ad', 'Wide'
        else:
            side, zone = 'Deuce', 'Wide'
    elif (-105 <= x <= -52.5) or (52.5 <= x <= 105):
        if sign > 0:
            side, zone = 'Ad', 'Body'
        else:
            side, zone = 'Deuce', 'Body'
    elif -52.5 < x < 52.5:
        if sign > 0:
            side, zone = 'Ad', 'T'
        else:
            side, zone = 'Deuce', 'T'
    else:
        side, zone = np.nan, np.nan
    
    return pd.Series({'side': side, 'Zone': zone})

In [ ]:
def favorite_serve(df_shots, df_points):

    # df_shots = df_shots[df_shots['__source_file__'].isin(df_points['__source_file__'])] # UPDATE: Temporary fix

    combined = pd.merge(df_shots, df_points[['Point', 'Game', 'Set', 'Point Winner', 'Match Server', '__source_file__']], on=['Point', 'Game', 'Set', '__source_file__'], how='left')

    serves = combined[(combined['Stroke'] == 'Serve') & (combined['Match Server'] == 'host')] # Added Player Name Filter
    serves_in = serves[serves['Result'] == 'In'].copy()

    serves_in.loc[:, 'x_coord'] = serves_in['Bounce (x)'] * 38.2764654418
    serves_in.loc[:, 'y_coord'] = (serves_in['Bounce (y)'] - 11.8872) * 38.2764654418
    serves_in[['side', 'serve_zone']] = serves_in.apply(classify_zone, axis=1)
    favorite_serve = serves_in['serve_zone'].value_counts().idxmax()

    return favorite_serve

In [ ]:
favorite_serve = favorite_serve(combined_data_shots, combined_data_points)
favorite_serve

### Best Serve

In [ ]:
def best_serve(df_shots, df_points, serve_type):
    # only use matches with complete data
    # df_shots = df_shots[df_shots['__source_file__'].isin(df_points['__source_file__'])] # UPDATE: Temporary fix

    # add column for winner of the point
    combined = pd.merge(df_shots, df_points[['Point', 'Game', 'Set', 'Point Winner', 'Match Server', '__source_file__']], on=['Point', 'Game', 'Set', '__source_file__'], how='left')

    serves = combined[(combined['Stroke'] == 'Serve') & (combined['Match Server'] == 'host')] # Added Player Name Filter
    serves_in = serves[serves['Result'] == 'In'].copy()

    # zone classification
    serves_in.loc[:, 'x_coord'] = serves_in['Bounce (x)'] * 38.2764654418
    serves_in.loc[:, 'y_coord'] = (serves_in['Bounce (y)'] - 11.8872) * 38.2764654418
    serves_in[['side', 'serve_zone']] = serves_in.apply(classify_zone, axis=1)

    # Subset by First or Second Serve
    serves_in = serves_in[(serves_in['Type'] == serve_type) & (serves_in['Point Winner'] == 'host')]
    best_serve = serves_in['serve_zone'].value_counts().idxmax()
    return best_serve

In [ ]:
best_first_serve = best_serve(combined_data_shots, combined_data_points, 'first_serve')
best_second_serve = best_serve(combined_data_shots, combined_data_points, 'second_serve')
best_first_serve, best_second_serve

### Serve Zone Distribution Bars

##### Helper Function

In [ ]:
def classify_zone(df):
    x = df['x_coord']
    y = df['y_coord']
    sign = x * y # if sign is pos, it's on ad side, if neg, it's deuce

    if (x < -105) or (x > 105):
        if sign > 0:
            side, zone = 'Ad', 'Wide'
        else:
            side, zone = 'Deuce', 'Wide'
    elif (-105 <= x <= -52.5) or (52.5 <= x <= 105):
        if sign > 0:
            side, zone = 'Ad', 'Body'
        else:
            side, zone = 'Deuce', 'Body'
    elif -52.5 < x < 52.5:
        if sign > 0:
            side, zone = 'Ad', 'T'
        else:
            side, zone = 'Deuce', 'T'
    else:
        side, zone = np.nan, np.nan
    
    return pd.Series({'side': side, 'Zone': zone})

In [ ]:
def serve_zone_bar(df_shots, df_points):
    df = pd.merge(df_shots, df_points[['Point', 'Game', 'Set', 'Point Winner', 'Match Server', '__source_file__']], on=['Point', 'Game', 'Set', '__source_file__'], how='left')

    serves = df[(df['Match Server'] == 'host') & (df['Type'].isin(['first_serve', 'second_serve']))]    

    # Keep only Serves In
    serves_in = serves[serves['Result'] == 'In'].copy()

    # Transform Coordinates
    serves_in.loc[:, 'x_coord'] = serves_in['Bounce (x)'] * 38.2764654418
    serves_in.loc[:, 'y_coord'] = (serves_in['Bounce (y)'] - 11.8872) * 38.2764654418

    # Create "side" and "serve_zone" columns
    serves_in[['side', 'serve_zone']] = serves_in.apply(classify_zone, axis=1)


    serves_in[['Type', 'side', 'serve_zone']].value_counts()

    # Step 1: Get the counts
    counts = serves_in[['Type', 'side', 'serve_zone']].value_counts().reset_index(name='count')

    # Step 2: Initialize output structure
    output = {'serves': []}

    # Step 3: Iterate through serve types
    for serve_type in counts['Type'].unique():
        serve_dict = {
            'type': 'First Serve' if serve_type == 'first_serve' else 'Second Serve',
            'sides': {
                'deuce': {'wide': 0, 'body': 0, 't': 0},
                'ad': {'wide': 0, 'body': 0, 't': 0}
            }
        }
        
        # Filter for this serve type
        subset = counts[counts['Type'] == serve_type]
        
        for _, row in subset.iterrows():
            side = row['side'].lower()     # 'Deuce' -> 'deuce'
            zone = row['serve_zone'].lower()  # 'Wide' -> 'wide'
            serve_dict['sides'][side][zone] = int(row['count'])
        
        output['serves'].append(serve_dict)

    # Save to current directory
    with open('data/serve_zones.json', 'w') as f:
        json.dump(output, f, indent=4)

In [ ]:
serve_zone_bar(combined_data_shots, combined_data_points)

### Serve Zone Distribution Court Visual

In [ ]:
def serve_placement_labels(df_shots, df_points, serve_type):
    # only use matches with complete data
    # df_shots = df_shots[df_shots['__source_file__'].isin(df_points['__source_file__'])] # UPDATE: Temporary fix

    # add column for winner of the point
    combined = pd.merge(df_shots, df_points[['Point', 'Game', 'Set', 'Point Winner', 'Match Server', '__source_file__']], on=['Point', 'Game', 'Set', '__source_file__'], how='left')

    # serves = combined[(combined['Stroke'] == 'Serve') & (combined['Match Server'] == 'host')] # Added Player Name Filter (CHANGED)
    serves = combined[(combined['Type'].isin(['first_serve', 'second_serve'])) & (combined['Match Server'] == 'host')] # Added Player Name Filter
    serves_in = serves[serves['Result'] == 'In'].copy()

    # zone classification
    serves_in.loc[:, 'x_coord'] = serves_in['Bounce (x)'] * 38.2764654418
    serves_in.loc[:, 'y_coord'] = (serves_in['Bounce (y)'] - 11.8872) * 38.2764654418
    serves_in[['side', 'serve_zone']] = serves_in.apply(classify_zone, axis=1)

    # Subset by First or Second Serve
    serves_in = serves_in[(serves_in['Type'] == serve_type)]
    
    # Step 1: Get the counts for each side/zone combination
    # Chat Gpt this shi
    counts = serves_in[['side', 'serve_zone', 'Point Winner']].value_counts().reset_index(name='count')
    
    # Step 2: Group by side and zone, calculate total serves and number of wins
    summary = (
        counts
        .groupby(['side', 'serve_zone'])
        .apply(lambda df: pd.Series({
            'total': df['count'].sum(),
            'won': df[df['Point Winner'] == 'host']['count'].sum()
        }))
        .reset_index()
    )

    # Step 3: Add win percentage column
    summary['win_percentage'] = (summary['won'] / summary['total'] * 100).round(1)

    # Step 4: Extract into variables
    # Initialize dictionaries to hold count and win %
    zone_counts = {}
    zone_win_percentages = {}

    for _, row in summary.iterrows():
        key = f"{row['side'].lower()}_{row['serve_zone'].lower()}"
        zone_counts[key] = row['total']
        zone_win_percentages[key] = row['win_percentage']

    return zone_counts, zone_win_percentages

In [ ]:
# First Serve
first_serve_zone_counts, first_serve_zone_win_percentages = serve_placement_labels(combined_data_shots, combined_data_points, 'first_serve')
first_serve_zone_counts, first_serve_zone_win_percentages

In [ ]:
# Second Serve
second_serve_zone_counts, second_serve_zone_win_percentages = serve_placement_labels(combined_data_shots, combined_data_points, 'second_serve')
second_serve_zone_counts, second_serve_zone_win_percentages

### Serve Placement (WIP)

##### Helper Functions

In [ ]:
# splits zone into two columns
def classify_zone_split(df):
    x = df['x']
    y = df['y']
    sign = x * y # if sign is pos, it's on ad side, if neg, it's deuce

    if (x < -105) or (x > 105):
        if sign > 0:
            return pd.Series(['Ad', 'Wide'])
        else:
            return pd.Series(['Deuce', 'Wide'])
    elif (-105 <= x <= -52.5) or (52.5 <= x <= 105):
        if sign > 0:
            return pd.Series(['Ad', 'Body'])
        else:
            return pd.Series(['Deuce', 'Body'])
    elif -52.5 < x < 52.5:
        if sign > 0:
            return pd.Series(['Ad', 'T'])
        else:
            return pd.Series(['Deuce', 'T'])
    else:
        return pd.Series([np.nan, np.nan])

In [ ]:
def generate_placement_jsons(df_shots, df_points, serve_type):
    # only use matches with complete data
    df_shots = df_shots[df_shots['__source_file__'].isin(df_points['__source_file__'])]

    # add column for winner of the point
    combined = pd.merge(df_shots, df_points[['Point', 'Game', 'Set', 'Point Winner', 'Match Server', 'Detail', '__source_file__']], on=['Point', 'Game', 'Set', '__source_file__'], how='left')

    serves = combined[(combined['Type'] == serve_type) & (combined['Match Server'] == 'host')]
    serves_in = serves[serves['Result'] == 'In'].copy()

    # zone classification
    serves_in.loc[:, 'x'] = serves_in['Bounce (x)'] * 38.2764654418
    serves_in.loc[:, 'y'] = (serves_in['Bounce (y)'] - 11.8872) * 38.2764654418
    serves_in[['side', 'serveInPlacement']] = serves_in.apply(classify_zone_split, axis=1)

    # modify coordinates based on the y-value
    serves_in['x'] = np.where(serves_in['y'] < 0, -serves_in['x'], serves_in['x'])
    serves_in['y'] = np.where(serves_in['y'] < 0, -serves_in['y'], serves_in['y'])

    # add serve outcome
    serves_in['serveOutcome'] = serves_in['Point Winner'].apply(lambda x: 'Won' if x == 'host' else 'Lost')
    serves_in['serveOutcome'] = np.where(serves_in['Detail'] == 'Ace', 'Ace', serves_in['serveOutcome'])

    # rename some columns to match json
    serves_in = serves_in.rename(columns={'Point': 'pointNumber', 'Player': 'serverName'})

    # only get required columns
    placement = serves_in[['pointNumber', 'serverName', 'x', 'y', 'side', 'serveInPlacement', 'serveOutcome']]
    placement.to_json(f'data/{serve_type}_place.json', orient='records') # save json


    ### LABELS JSON ###

    # group by side and serveInPlacement, and calculate count and serves won
    distribution = serves_in.groupby(['side', 'serveInPlacement']).agg(
        count=('pointNumber', 'size'),
        serves_won=('Point Winner', lambda x: (x == 'host').sum())
    ).reset_index()

    # calculate the win percentage (proportion)
    distribution['proportion'] = distribution['serves_won'] / distribution['count']

    # find the min and max proportions
    min_proportion = distribution['proportion'].min()
    max_proportion = distribution['proportion'].max()

    # create labels df and determine if each value is max, min, or neither
    labels = distribution.copy()
    labels['proportion_label'] = (labels['proportion'] * 100).round(1).astype(str) + "%"
    labels['count_label'] = labels['count']

    labels['x'] = np.where(
        (labels['side'] == 'Ad') & (labels['serveInPlacement'] == 'Wide'), 131.25,
        np.where(
            (labels['side'] == 'Ad') & (labels['serveInPlacement'] == 'Body'), 78.75,
            np.where(
                (labels['side'] == 'Ad') & (labels['serveInPlacement'] == 'T'), 26.25,
                np.where(
                    (labels['side'] == 'Deuce') & (labels['serveInPlacement'] == 'T'), -26.25,
                    np.where(
                        (labels['side'] == 'Deuce') & (labels['serveInPlacement'] == 'Body'), -78.75,
                        np.where(
                            (labels['side'] == 'Deuce') & (labels['serveInPlacement'] == 'Wide'), -131.25,
                            np.nan
                        )
                    )
                )
            )
        )
    )

    # determine max/min status
    labels['max_min'] = np.where(
        labels['proportion'] == max_proportion, "max",
        np.where(labels['proportion'] == min_proportion, "min", "no")
    )

    # save labels to json
    labels.to_json(f'data/{serve_type}_place_labels.json', orient='records')

    return

In [ ]:
generate_placement_jsons(combined_data_shots, combined_data_points, 'first_serve')
generate_placement_jsons(combined_data_shots, combined_data_points, 'second_serve')

### Serve Ratings

##### Helper Functions

In [ ]:
# Helper Function: Grabs total of Specified Stat
def get_total(df, stat_name):
    rows = df[df['Stat Name'] == stat_name]
    if rows.empty: # UPDATE Throw an error?
        return 0
    total = 0
    for col in rows.columns:
        if col.startswith('Host Set'):
            numeric_vals = pd.to_numeric(rows[col], errors='coerce')
            total += numeric_vals.sum()  # Ignores NaNs automatically # UPDATE: throw an error? because this means that the data is bad?
    return total
    # return rows

In [ ]:
# Helper Function: Calculate Service Games Won Percentage

def calculate_service_games_won(df):
    # Filter the DataFrame for host server and non-draw game winner
    host_service_games = df[(df['Server'] == 'host') & (df['Game Winner'] != 'draw')]
    
    # Filter for games where host won
    host_service_games_won = host_service_games[host_service_games['Game Winner'] == 'host']
    
    service_games_won_percentage = len(host_service_games_won) / len(host_service_games) 
    
    return service_games_won_percentage

In [ ]:
# Helper Function: Calculate Double Faults

def calculate_double_faults(df):
    double_fault_total = df[(df['Match Server'] == 'host') & 
                                   (df['Detail'] == 'Double Fault')].shape[0]
    return double_fault_total

##### Serve Rating Calculation

In [ ]:
# Total Matches in dataset
total_matches = len(combined_data_stats.groupby('__source_file__'))

# 1st Serve In Percentage
first_serve_in_percentage_rating = ((get_total(combined_data_stats, '1st Serves In') / get_total(combined_data_stats, '1st Serves')) * 100).round(1)

# 1st Serve Points Won Percentage
first_serve_won_percentage_rating = ((get_total(combined_data_stats, '1st Serves Won') / get_total(combined_data_stats, '1st Serves In')) * 100).round(1)

# 2nd Serve Points Won Percentage
second_serve_won_percentage_rating = ((get_total(combined_data_stats, '2nd Serves Won') / get_total(combined_data_stats, '2nd Serves In')) * 100).round(1)

# Service Games Won Percentage
service_games_won_percentage_rating = round(calculate_service_games_won(combined_data_games) * 100, 1)

# Average Aces per Match Percentagae
aces_average_rating = ((get_total(combined_data_stats, 'Aces') / total_matches)).round(1)

# Average Double Faults per Match Percentage
doubleFaults_average_rating = round(calculate_double_faults(combined_data_points) / total_matches, 1)

In [ ]:
# Calculate Serve Rating
serve_rating = round(first_serve_in_percentage_rating + first_serve_won_percentage_rating + second_serve_won_percentage_rating + service_games_won_percentage_rating + aces_average_rating - doubleFaults_average_rating, 1)

# Print All Calculations
print(f"First Serve In %: {first_serve_in_percentage_rating}%")
print(f"First Serve Points Won %: {first_serve_won_percentage_rating}%")
print(f"Second Serve Points Won %: {second_serve_won_percentage_rating}%")
print(f"Service Games Won %: {service_games_won_percentage_rating}%")
print(f"Aces per Match: {aces_average_rating}")
print(f"Double Faults per Match: {doubleFaults_average_rating}")
print(f"Serve Rating: {serve_rating}")

### Output CSV

In [ ]:
# Formatted ratings
first_serve_in_percentage_rating_updated = '+' + str(first_serve_in_percentage_rating) + '%'
first_serve_won_percentage_rating_updated = '+' + str(first_serve_won_percentage_rating) + '%'
second_serve_won_percentage_rating_updated = '+' + str(second_serve_won_percentage_rating) + '%'
service_games_won_percentage_rating_updated = '+' + str(service_games_won_percentage_rating) + '%'
aces_average_rating_updated = '+' + str(aces_average_rating)
doubleFaults_average_rating_updated = '-' + str(doubleFaults_average_rating)

In [ ]:
serve_summary = {
    "fastest_serve": [fastest_serve],
    "favorite_serve": favorite_serve,
    "breakpoints_saved": breakpoints_saved_percentage,
    "min": min,
    "sec": sec,
    "first_serve_in": first_serve_in_percentage_rating_updated,
    "first_serve_won": first_serve_won_percentage_rating_updated,
    "second_serve_won": second_serve_won_percentage_rating_updated,
    "service_games_won": service_games_won_percentage_rating_updated,
    "average_aces": aces_average_rating_updated,
    "average_double_faults": doubleFaults_average_rating_updated,
    "serve_rating": serve_rating,
    "total_serves_won": total_serve_points_won
}

df = pd.DataFrame(serve_summary)


# Output to CSV
output_path = f"data/{player_name.replace(' ', '')}_serve0.csv"
df.to_csv(output_path, index=False)

In [ ]:
first_serve_summary = {
    "first_serve_in": str(first_serve_in_percentage) + '%',
    "first_serve_points_won": str(first_serve_won_percentage) + '%',
    "best_first_serve": [best_first_serve],
    "average_aces": average_aces,
    "ad_body_count": first_serve_zone_counts['ad_body'],
    "ad_t_count": first_serve_zone_counts['ad_t'],
    "ad_wide_count": first_serve_zone_counts['ad_wide'],
    "deuce_body_count": first_serve_zone_counts['deuce_body'],
    "deuce_t_count": first_serve_zone_counts['deuce_t'],
    "deuce_wide_count": first_serve_zone_counts['deuce_wide'],
    "ad_body_won": str(first_serve_zone_win_percentages['ad_body']) + '%',
    "ad_t_won": str(first_serve_zone_win_percentages['ad_t']) + '%',
    "ad_wide_won": str(first_serve_zone_win_percentages['ad_wide']) + '%',
    "deuce_body_won": str(first_serve_zone_win_percentages['deuce_body']) + '%',
    "deuce_t_won": str(first_serve_zone_win_percentages['deuce_t']) + '%',
    "deuce_wide_won": str(first_serve_zone_win_percentages['deuce_wide']) + '%'
}

df = pd.DataFrame(first_serve_summary)

# Output to CSV
output_path = f"data/{player_name.replace(' ', '')}_serve1.csv"
df.to_csv(output_path, index=False)

In [ ]:
second_serve_summary = {
    "second_serve_in": str(second_serve_in_percentage) + '%',
    "second_serve_points_won": str(second_serve_won_percentage) + '%',
    "best_second_serve": [best_second_serve],
    "average_double_faults": average_double_faults,
    "ad_body_count": second_serve_zone_counts['ad_body'],
    "ad_t_count": second_serve_zone_counts['ad_t'],
    "ad_wide_count": second_serve_zone_counts['ad_wide'],
    "deuce_body_count": second_serve_zone_counts['deuce_body'],
    "deuce_t_count": second_serve_zone_counts['deuce_t'],
    "deuce_wide_count": second_serve_zone_counts['deuce_wide'],
    "ad_body_won": str(second_serve_zone_win_percentages['ad_body']) + '%',
    "ad_t_won": str(second_serve_zone_win_percentages['ad_t']) + '%',
    "ad_wide_won": str(second_serve_zone_win_percentages['ad_wide']) + '%',
    "deuce_body_won": str(second_serve_zone_win_percentages['deuce_body']) + '%',
    "deuce_t_won": str(second_serve_zone_win_percentages['deuce_t']) + '%',
    "deuce_wide_won": str(second_serve_zone_win_percentages['deuce_wide']) + '%'
}

df = pd.DataFrame(second_serve_summary)

# Output to CSV
output_path = f"data/{player_name.replace(' ', '')}_serve2.csv"
df.to_csv(output_path, index=False)